# Installation
Just pip install:

```pip install omegaconf```

# Basic example
We will use this simple example.yaml file:

In [ ]:
%load 'docs/source/example.yaml'

## Loading a file

In [2]:
from omegaconf import OmegaConf
conf = OmegaConf.from_filename('docs/source/example.yaml')
conf

{'server': {'port': 80}, 'log': {'file': 'log.txt', 'rotation': 3600}}

## Reading values

In [3]:
# Object style access
conf.server.port

80

In [4]:
# Map style access
conf['log']

{'file': 'log.txt', 'rotation': 3600}

## Reading with default values:
You can easily provide a default value to use if the object is missing a key:

In [5]:
conf.missing_key or 'a default value'

'a default value'

In [6]:
conf.get('missing_key', 'a default value')

'a default value'

## Manipulating config:

In [7]:
# Changing existing keys
conf.server.port = 81
# Adding new keys
conf.server.hostname = "localhost"
# Or new sections
conf.database = {'hostname': 'database01', 'port': 3306}

# Creating OmegaConf objects
## Empty config

In [8]:
conf = OmegaConf.empty()
conf.a = {'key' :'value'}
conf

{'a': {'key': 'value'}}

## From filename

In [9]:
conf = OmegaConf.from_filename('docs/source/example.yaml')
conf

{'server': {'port': 80}, 'log': {'file': 'log.txt', 'rotation': 3600}}

## From file

In [10]:
import io
file = io.open('docs/source/example.yaml', 'r')
conf = OmegaConf.from_file(file)
conf

{'server': {'port': 80}, 'log': {'file': 'log.txt', 'rotation': 3600}}

## From CLI arguments

from_cli() will parse anything in sys.argv. Note that if you want to use this with a CLI parser, it will have to clear anything it already parsed from sys.argv before you initialize the conf from_cli().
You can also pass a list of arguments directly.

You can use dot notation to create nested elements.

In [11]:
# Parse sys.argv:
conf = OmegaConf.from_cli()

# Parse given list:
arguments = ['key=value', 'parent.child=2']
conf = OmegaConf.from_cli(arguments)
conf

{'key': 'value', 'parent': {'child': 2}}

In [12]:
yaml_string = """
yaml_string: can
be:
  - loaded
  - directly
"""
conf = OmegaConf.from_string(yaml_string)
conf

{'yaml_string': 'can', 'be': ['loaded', 'directly']}

# Layering OmegaConf objects
OmegaConf can be created from files, from cli from strings etc.

You can combine configurations, regardless of their source.

Here are a few examples where you might want to merge different configurations:

* You want to use more than one configuration file but access them uniformly through the same object
* You want to override some configuration in a file with that from another file
* You want to override configuration using command line arguments

We will use two configuration files:

In [ ]:
%load 'docs/source/example.yaml'

In [ ]:
%load 'docs/source/example2.yaml'

We will overlay example2.yaml on top of example.yaml, combining them and overridng example.yaml when there is a conflict:

In [15]:
conf1 = OmegaConf.from_filename('docs/source/example.yaml')
conf2 = OmegaConf.from_filename('docs/source/example2.yaml')
conf = OmegaConf.merge(conf1, conf2)
conf.server.port

81

In [16]:
conf.database

{'port': 3306, 'host': 'dbhost'}

# Merging file with CLI args
Now let’s try overriding conf1 above with parameters from the command line: For simplicity, we will simulate command line arguments by setting sys.argv.

In [17]:
import sys
conf1 = OmegaConf.from_filename('docs/source/example.yaml')
# Simulate command line arguments
sys.argv = ['program.py', 'server.port=82', 'log.file=log2.txt']
cli = OmegaConf.from_cli()
conf = OmegaConf.merge(conf1, cli)
conf.server.port

82

In [18]:
conf.log.file

'log2.txt'

# Putting it all together
OmegaConf.merge() can take any number of configuration objects, layering them one on top of the other in order.
For example, you could load two files, layer them one on top of the other, followed by the CLI arguments:

In [19]:
conf1 = OmegaConf.from_filename('docs/source/example.yaml')
conf2 = OmegaConf.from_filename('docs/source/example2.yaml')
sys.argv = ['program.py', 'server.port=9999', 'log.file=log2.txt']
cli = OmegaConf.from_cli()
conf = OmegaConf.merge(conf1, conf2, cli)
conf

{'server': {'port': 9999}, 'log': {'file': 'log2.txt', 'rotation': 3600}, 'database': {'port': 3306, 'host': 'dbhost'}}